# Decay - simple working example

This notebook tests a simple decay process.

In [1]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

Potential organ recipients can be added (i.e. to a waiting list). We expect that an update on a potential recipient will be received within 150 time periods. The hospital may want to check that this patient is still alive, does not have any new diseases the hospital is unaware of, and so on.

We monitor the age of waiting records using multi valued fluents. The age of records is set to 1 when the waiting fluent is initiated by a recipient event. We create an expectation rule that a record will be renewed before the waiting record is 150 time periods old. We use a decay event, which occurs at every time period, to initiate updates to waiting record ages.

In [2]:
% File: initiates.pl
happensAtNarrative(decay,_).
initiates(addToList(RID,PatID),waiting(RID,PatID)=1,_).
initiates(renew(PatID), renewed(PatID),_).
initially(exp_rule(waiting(RID,PatID)=1, before(renewed(PatID), waiting(RID,PatID)=150))).
initiates(decay, waiting(RID,PatID)=Y,T):- holdsAt(waiting(RID,PatID)=X,T), Y is X + 1.

We create a recipient record for patient 123 at time zero, and a record for patient 124 at time four. Information for patient 124 is renewed at time 50.

In [3]:
% File: narrative.pl
happensAtNarrative(addToList(3,123),0).
happensAtNarrative(addToList(4,124),4).
happensAtNarrative(renew(124),50).

At time zero, patient 123 is added to the wait list.

In [4]:
?- tick(0).
?- holdsAt(F,0).
?- happensAt(E,0).

true.
F = exp_rule(=(waiting(_3040, _3042), 1), before(renewed(_3042), =(waiting(_3040, _3042), 150))) .
E = decay ;
E = addToList(3, 123) .

We can see below that at time one, patient 123 has been waiting for 1 time period.

In [5]:
?- tick(1).
?- holdsAt(F,1).

true.
F = {'waiting(3, 123)': 1} ;
F = exp_rule(=(waiting(_3020, _3022), 1), before(renewed(_3022), =(waiting(_3020, _3022), 150))) ;
F = exp(before(renewed(123), =(waiting(3, 123), 150))) ;
F = exp(=(waiting(3, 123), 1), before(renewed(123), =(waiting(3, 123), 150)), 1, before(renewed(123), =(waiting(3, 123), 150))) .

At time 2, patient 123 has been waiting for 2 time periods.

In [6]:
?- tick(2).
?- holdsAt(F,2).

true.
F = {'waiting(3, 123)': 2} ;
F = exp_rule(=(waiting(_3020, _3022), 1), before(renewed(_3022), =(waiting(_3020, _3022), 150))) ;
F = exp(before(renewed(123), =(waiting(3, 123), 150))) ;
F = exp(=(waiting(3, 123), 1), before(renewed(123), =(waiting(3, 123), 150)), 1, before(renewed(123), =(waiting(3, 123), 150))) .

At time 50, information for patient 124 is renewed, and the expectation of renewal within 150 time periods of the recipient record being made is fulfilled.

In [7]:
?- initialiseDEC.
?- run(50).
?- happensAt(B,50).
?- tick(51).
?- happensAt(D,51).

true.
true.
B = decay ;
B = renew(124) .
true.
D = decay ;
D = fulf(=(waiting(4, 124), 1), before(renewed(124), =(waiting(4, 124), 150)), 5, before(renewed(124), =(waiting(4, 124), 150))) .

At time 150, information for patient 123 has not been renewed and the expectation is violated.

In [8]:
?- initialiseDEC.
?- run(150).
?- holdsAt(waiting(RID,123)=Age,150).
?- happensAt(B,150).

true.
true.
RID = 3, Age = 150 .
B = decay ;
B = viol(=(waiting(3, 123), 1), before(renewed(123), =(waiting(3, 123), 150)), 1, before(renewed(123), =(waiting(3, 123), 150))) .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>